In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time
import random

import Init_NetRate as Init
import cvxpy as cp
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx

In [68]:
'''
Evaluate the objective function given a cascade and the alphas

Input :
    -alpha_mat : matrix of size |V|*|V| containing all the alpha_i,j
    -cascade_graph : Directed graph object (DAG), of a cascade
    -G_star : Directed graph containing only the nodes
    -window : int representing the time limit until which we observe a cascade propagation
Output :
    -obj : value of the objective funtion restricted to one cascade
'''
def Objective_function(alpha_mat,cascade_graph_c,G_star,window,eps):
    sum_psi1 = 0
    sum_psi2 = 0
    sum_psi3 = 0
    for i in cascade_graph_c.nodes():
        sum_psi3_tmp = 0
        t_i = cascade_graph_c.nodes[i]["time"]
        for m in G_star.nodes():
            if alpha_mat[i,m]==eps :
                alpha_mat[i,m] = 0
            if alpha_mat[m,i]==eps:
                alpha_mat[m,i] = 0
            if m not in cascade_graph_c.nodes():
                sum_psi1 += -alpha_mat[i,m]*(window-t_i)
            elif (m,i) in cascade_graph_c.edges():
                t_m = cascade_graph_c.nodes[m]["time"]
                sum_psi2 += -alpha_mat[m,i]*(t_i-t_m)
#                 if alpha_mat[m,i]==0:
#                     sum_psi3_tmp+=eps
#                 else:
#                     sum_psi3_tmp += alpha_mat[m,i]
                sum_psi3_tmp += alpha_mat[m,i]
        if sum_psi3_tmp!=0:
            sum_psi3 += math.log(sum_psi3_tmp)
        else :
            if len(list(cascade_graph_c.predecessors(i)))>0:
                print("log of zero")
#                 sum_psi3 = -1e10 # should mimic the - infinity
                sum_psi3 += math.log(eps)
    obj = sum_psi1+sum_psi2+sum_psi3
    return obj


def True_objective_value(G_true,cascade_graph_dic,window,eps):
    A_true = np.zeros((G_true.number_of_nodes(),G_true.number_of_nodes()))
    for edge in G_true.edges():
        try :
            A_true[edge[0],edge[1]] = G_true.edges[edge[0],edge[1]]["weight"][0]
        
        except TypeError:
            A_true[edge[0],edge[1]] = G_true.edges[edge[0],edge[1]]["weight"]
            
    true_obj = 0
    for c in cascade_graph_dic:
        cascade = cascade_graph_dic[c]
        true_obj += Objective_function(A_true,cascade,G_true,window,eps)
    return -true_obj
def Infered_objective_value(alpha_mat,G_star,cascade_graph_dic,window,eps):
    obj = 0
    for c in cascade_graph_dic:
        cascade = cascade_graph_dic[c]
        obj += Objective_function(alpha_mat,cascade,G_star,window,eps)
    return -obj

In [69]:
obj = Infered_objective_value(A_hat,G_star,DAG_C_dic,window,eps)
obj

log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log of zero
log 

20048.167787540366

In [66]:
true_obj = True_objective_value(G_true,DAG_C_dic,window,eps)
true_obj

17175.029425916546

In [2]:
'''
Create the matrix Mi for the ADMM method

Each cell k,l in Mi is one if in cascade k node l infected node i, 0 otherwise

Input :
    cascade_dic : a dictionnary of all cascade (in a graph object, DAG)
    node : an int describing which node we are considering in the ADMM iteration
    number_of_nodes : number of nodes in the underlying network (we assume that rhe union of all cascades cover all the nodes)
'''
def Create_matrix_Mi_and_Ti(cascade_dic,node,number_of_nodes,window) : 
    M_i = np.zeros((len(cascade_dic),number_of_nodes))
    T_i = np.zeros((len(cascade_dic),number_of_nodes))
    for cascade in cascade_dic :
        c = cascade_dic[cascade] # graph object
        if node in c.nodes():
            t_i = c.node[node]["time"]
            parent_list = list(c.predecessors(node)) # create a list of all nodes (int) that where infected before node i in the cascade
            for parent in parent_list :
                t_parent = c.node[parent]["time"]
                M_i[cascade,parent] = 1
                T_i[cascade,parent] = (t_i-t_parent)
                if (t_i-t_parent)<=0:
                    print("Time error, the flow of time is reversed the world's end is near")
        else :
            for j in c.nodes() :
                t_j = c.node[j]["time"]
                T_i[cascade,j] = (window-t_j) # SIGNE is now correct !
                
    return M_i,T_i


    

In [70]:
def Compute_objective_value_for_one_node(alpha_mat_i,cascade_dic,window,eps,node_i) :
    obj_i = 0
    for cascs in cascade_dic :
        c = cascade_dic[cascs]
        if node_i in c.nodes():
            t_i = c.nodes[node_i]["time"]
            parents_i_list = list(c.predecessors(node_i))
            sum_tmp = 0
            for parent in parents_i_list :
                t_parent = c.nodes[parent]["time"]
                obj_i += alpha_mat_i[parent]*t_parent
                sum_tmp += alpha_mat_i[parent]
            if sum_tmp<=0 :
                if len(parents_i_list)!=0 :
                    print("Attention log ill defined incoming")
                    obj_i -=math.log(eps)
            else :
                obj_i -= math.log(sum_tmp)
        else :
            for j in c.nodes():
                t_j = c.nodes[j]["time"]
                obj_i += alpha_mat_i[j]*(window-t_j)
    return obj_i

In [71]:
total_obj = 0
obj_per_node = []
for i in G_true.nodes():
    alpha_mat_i = A_hat[:,i]
    obj_i = Compute_objective_value_for_one_node(alpha_mat_i,DAG_C_dic,window,eps,i)
    print("Node %i has objective value %f" %(i,obj_i))
    obj_per_node.append(obj_i)
    total_obj += obj_i
    

Node 0 has objective value 1832.775828
Node 1 has objective value 218.378587
Node 2 has objective value 0.000000
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention lo

Node 16 has objective value 391.181318
Node 17 has objective value 1625.571451
Node 18 has objective value 1029.736674
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Node 19 has objective value 1440.662377
Node 20 has objective value 1701.217478
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incoming
Attention log ill defined incom

In [72]:
total_obj

29313.679097536533

In [30]:
network_file_name = "./Graph_test_SG.txt"
cascade_file_name = "./Cascade_test_SG.txt"
window = 10
model=0 # not important but needed. When constructing the underlying network specify that we use the exponential law
beta = 1 # used for the construction of the cascades
eps = 0.0005
alpha_max = 10
alpha_init = 0.01
iter_ADMM = 20 #number of itteration for 1 node in the ADMM method. This is a parameter to tune
iter_GD = 5000
gamma = 0.00001 # Learning rate of the GD for alpha


In [4]:
# G_true = Init.Load_ground_truth(network_file_name)
# G_star, DAG_C_dic = Init.Init(cascade_file_name)
# N = G_true.number_of_nodes()

G_true = Gen.Generate_random_graph(30,60)
Cascades = Gen.Generate_all_cascades(G_true,-1000,window,model,beta)
Gen.Save_cascade_to_file("Cascade_test_ADMM.txt",Cascades,G_true)
Gen.Save_graph_to_file("Graph_test_ADMM.txt",G_true)
G_star,DAG_C_dic = Init.Init("Cascade_test_ADMM.txt")
N = G_true.number_of_nodes()

All nodes were read


In [29]:
# (A_pot,A_bad),num_casc_per_node = cvx.Create_matrices_ADMM(G_true,DAG_C_dic,window)
A_hat = np.zeros((N,N))
u = 1 # used for the gradient descent of rho and as a penalizer and the constrain


In [31]:
t_start_global = time.time()
dic_of_obj_per_node_per_iter = {}
obj_per_node = []
for i in G_true.nodes :
    print("Node : ",i)
    t_start_node_i = time.time()
    dic_of_obj_per_node_per_iter[i] = []
#     try :
#         cascade_list = num_casc_per_node[i]
#     except KeyError : # node i was present in no cascade hence all there should be no edge connection to it
#         A_hat[:,i] =0
#         continue
    
    
    '''
    initialization
    '''
#     a_k = np.random.rand(N,1)
#     z = np.random.rand(N,1)
    a_k = np.ones((N,1))*alpha_init
    z = a_k.copy()
    rho = np.zeros((len(DAG_C_dic),1)) 
    M_i,T_i = Create_matrix_Mi_and_Ti(DAG_C_dic,i,N,window) # TO do : consider to use sparse matrix
    S_i = np.matmul(M_i,z)  
    grad_i = (np.sum(T_i,axis=0).T)
    
    
    #Start iteration of ADMM
    for k in range(0,iter_ADMM) :
        '''
        Update alpha using gradient descent
        '''
        grad = grad_i + np.matmul(rho.T,M_i)
        for j in range(0,iter_GD):
            grad_j = grad - u*(np.matmul(M_i.T,(S_i-np.matmul(M_i,a_k))).T) # sign is correct
            a_k = a_k - gamma*grad_j.T
            a_k = np.maximum(a_k,0)
            a_k = np.minimum(a_k,alpha_max)
        '''
        update S_i and rho for each cascades via the closed form formula and the gradient descent respectively
        '''
        for cascs in DAG_C_dic :
            c = DAG_C_dic[cascs]
#             Malpha = 0
#             if i in c.nodes :
#                 parent_i_c = list(c.predecessors(i))
#                 for papa in parent_i_c :
#                     Malpha += a_k[papa]
#             sqrt_delta = math.sqrt((rho[cascs]+Malpha)**2 + 4*u)
#             S_i[cascs] = ((rho[cascs]+Malpha)+sqrt_delta)/(2*u)
            Malpha = np.matmul(M_i[cascs,:],a_k)            
            sqrt_delta = math.sqrt((rho[cascs]+u*Malpha)**2+4*u)
            S_i[cascs] = (rho[cascs]+u*Malpha + sqrt_delta)/(2*u)
            if S_i[cascs]<0 :
                print("Huston Huston we have a probleme")
            rho[cascs] = rho[cascs]-u*(S_i[cascs]-Malpha)
            
#         '''compute the objective function for node i in iteration k'''
#         obj_i_k = 0
#         for c in DAG_C_dic :
#             cascade = DAG_C_dic[c]
#             sum_tmp = 0
#             for j in cascade.nodes():
#                 t_j = cascade.nodes[j]["time"]
#                 if a_k[j]<eps:
#                         a_k[j]=0
#                 if (j,i) in cascade.edges():
#                     t_i = cascade.node[i]["time"]
#                     obj_i_k += -a_k[j]*(t_i-t_j)
# #                     if a_k[j]<eps:
# #                         sum_tmp += eps
# #                     else:
# #                         sum_tmp += a_k[j]
#                     sum_tmp += a_k[j]
#                 if j in cascade.nodes() and i not in cascade.nodes():
#                     obj_i_k += -a_k[j]*(window-t_j)
#             if sum_tmp >0:
#                 obj_i_k += math.log(sum_tmp)
#         dic_of_obj_per_node_per_iter[i].append(obj_i_k)
    A_hat[:,i] = a_k.flatten()
#     obj_per_node.append(obj_i_k)
#     print("Total objective function value is : ",sum(obj_per_node))
    t_end_node_i = time.time()
    print("computation time for node i : ", t_end_node_i-t_start_node_i)
t_end_global = time.time()
print("total computation time : ",t_end_global-t_start_global) 

Node :  0
computation time for node i :  5.220898866653442
Node :  1
computation time for node i :  5.145010709762573
Node :  2
computation time for node i :  5.477331161499023
Node :  3
computation time for node i :  5.268018484115601
Node :  4
computation time for node i :  5.989699125289917
Node :  5
computation time for node i :  5.666798830032349
Node :  6
computation time for node i :  5.242227077484131
Node :  7
computation time for node i :  6.284820079803467
Node :  8
computation time for node i :  6.010034561157227
Node :  9
computation time for node i :  6.712867498397827
Node :  10
computation time for node i :  5.396979093551636
Node :  11
computation time for node i :  5.081522226333618
Node :  12
computation time for node i :  5.281905889511108
Node :  13
computation time for node i :  4.967442512512207
Node :  14
computation time for node i :  5.200066328048706
Node :  15
computation time for node i :  5.047299385070801
Node :  16
computation time for node i :  5.221394

In [32]:
'''
Precision and recall computation
'''
correct = 0
edge_G_star = list(G_star.edges())
G_star.remove_edges_from(edge_G_star)
for i in range(0,G_true.number_of_nodes()):
    for j in range(0,G_true.number_of_nodes()):
        if A_hat[i,j] >0:
            G_star.add_edge(i,j)
            if (i,j) in G_true.edges():
                correct +=1
#                 print("edge ",(i,j))
#                 print("alpha is ",A_hat[i,j])
print("Precision :",correct/G_star.number_of_edges())
print("Recall :",correct/G_true.number_of_edges())
print("Number of correct infered edges : ",correct)
print("Number of edges in G_hat :",G_star.number_of_edges())
print("Number of edges in the true network ",G_true.number_of_edges())
        

Precision : 0.5813953488372093
Recall : 0.8333333333333334
Number of correct infered edges :  50
Number of edges in G_hat : 86
Number of edges in the true network  60


In [28]:
'''
mean square error
'''
mse = 0
nb_correcte_edges=0
for edge in G_true.edges():
    if edge in G_star.edges():
        nb_correcte_edges +=1
        try:
            true_alpha = G_true.edges[edge]["weight"][0]
        except TypeError:
            true_alpha = G_true.edges[edge]["weight"]
        approx_alpha = A_hat[edge[0],edge[1]]
        mse += pow((true_alpha-approx_alpha),2) # mean square error
#         print("For edge " + str(edge)+ " the true weight is : " + str(true_alpha))
#         print("For edge " + str(edge)+ " the approx weight is : " + str(approx_alpha))
try :
    mse = mse/nb_correcte_edges
except ZeroDivisionError:
    print ("There is no correct edge")
print(" Mean square error is : ",mse)

 Mean square error is :  0.003909523029473034
